Account

In [3]:
# Loaded variable 'df' from URI: c:\Users\ohechihiro\Documents\d365-migration\cmt-files\formatted-data\account.parquet
import pandas as pd
df = pd.read_parquet(r'c:\Users\ohechihiro\Documents\d365-migration\cmt-files\formatted-data\account.parquet')

# Define a function to convert full-width to half-width
def to_half_width(text):
    if pd.isna(text):
        return text
    return text.translate(str.maketrans(
        "０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ　",
        "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "
    ))
# Apply the function to all object columns
df = df.applymap(lambda x: to_half_width(x) if isinstance(x, str) else x)



C:\Users\ohechihiro\AppData\Local\Temp\ipykernel_19360\196850196.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: to_half_width(x) if isinstance(x, str) else x)


In [27]:
# Save the updated DataFrame back to the same Parquet file
df.to_parquet(r'c:\Users\ohechihiro\Documents\d365-migration\cmt-files\formatted-data\account.parquet', engine='pyarrow')
print("DataFrame has been saved to account.parquet.")

DataFrame has been saved to account.parquet.


In [26]:
import requests
import pandas as pd

# secret.txtを読み込む関数
def load_secrets(file_path):
    secrets = {}
    with open(file_path, "r") as file:
        for line in file:
            # 行を分割してキーと値を取得
            key, value = line.strip().split("=", 1)
            secrets[key] = value
    return secrets

# secret.txtのパス
secrets_file = "secrets.txt"

# 機密情報を読み込む
secrets = load_secrets(secrets_file)

# Azure Maps APIキー
API_KEY = secrets.get("AZURE_MAPS_API_KEY")

# 同期型でAzure Maps APIにリクエストを送信する関数
def fetch_lat_lon(address):
    if not address:  # addressが値を持たない場合はリクエストを送信しない
        return None, None

    url = "https://atlas.microsoft.com/search/address/json"
    params = {
        "api-version": "1.0",
        "subscription-key": API_KEY,
        "query": address
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("results"):
                location = data["results"][0]["position"]
                return location["lat"], location["lon"]
        return None, None
    except Exception as e:
        print(f"Error fetching data for address '{address}': {e}")
        return None, None

# DataFrameの住所列を同期型で処理
addresses = df['address1_composite'].tolist()
results = [fetch_lat_lon(address) for address in addresses]

# DataFrameに結果を追加
df[['address1_latitude', 'address1_longitude']] = pd.DataFrame(results, columns=['latitude', 'longitude'])
print("Latitude and Longitude have been added to the DataFrame.")

Latitude and Longitude have been added to the DataFrame.


In [ ]:
df